# MODELADO

## HEURÍSTICA II


<hr>

<code> **Proyecto de Datos II** </code>

## Índice

- [Importación de los datos](#importación-de-los-datos)
- [Preprocesamiento](#preprocesamiento)
- [Entrenamiento](#entrenamiento)
- [Análisis del modelo](#análisis-del-modelo)
- [Registro del modelo en MLflow](#registro-del-modelo-en-mlflow)


In [3]:
import time
import mlflow
import pandas as pd
from evaluation.evaluator import Evaluator

SEED = 22 # replicabilidad

# =====================================
MODEL_NAME = "Heurística II"
# =====================================

## Importación de los datos

In [5]:
df_train = pd.read_parquet("../data/train.parquet")
df_test = pd.read_parquet("../data/test.parquet")

# ! NOTA -> están el ICAO, Callsign y Timestamp por si hay que depurar
X_train, y_train = df_train.drop(columns="takeoff_time", axis=1), df_train["takeoff_time"]
X_test, y_test = df_test.drop(columns="takeoff_time", axis=1), df_test["takeoff_time"]

In [6]:
X_train.shape, X_test.shape

((123733, 60), (27791, 60))

## Preprocesamiento

Para la heurística definida no será necesario ningún tipo de preprocesamiento.

## Entrenamiento

In [10]:
import pandas as pd

def h(df):
    df['date'] = df['timestamp'].dt.date
    df = df.sort_values(by='timestamp')

    # 3. Inicializar lista para las predicciones
    predictions = []

    # 4. Aplicar predicción a todo el DataFrame
    for idx, row in df.iterrows():
        # Filtrar los eventos anteriores en la misma pista, el mismo día y con callsign distinto
        mask = (
            (df['runway'] == row['runway']) & 
            (df['date'] == row['date']) & 
            (df['timestamp'] < row['timestamp']) & 
            (df['callsign'] != row['callsign']) & 
            (pd.to_timedelta(df['takeoff_time'], unit='s') + df['timestamp'] < row['timestamp'])
        )
        
        previous_events = df[mask]
        
        # Agrupar por callsign y tomar el mensaje más antiguo (primero) de cada avión
        grouped_events = previous_events.groupby('callsign').first().reset_index()

        # Tomar los 3 eventos más recientes de los aviones válidos
        recent_events = grouped_events.sort_values(by='timestamp', ascending=False).head(3)
        
        if not recent_events.empty:
            pred = recent_events['takeoff_time'].mean()  # Promedio de los 3 anteriores
        else:
            pred = 170  # Valor por defecto cuando no hay eventos válidos
        
        predictions.append(pred)

    # 5. Asignar las predicciones a la columna 'prediction' en el DataFrame
    df['prediction'] = predictions

    return df


In [11]:
start_time = time.time()

df_train = h(df_train)

end_time = time.time()
execution_time = end_time - start_time

## Análisis del modelo

In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# ===============================================================
y_true = df_train['takeoff_time']
y_pred = df_train['prediction']

mae_train = mean_absolute_error(y_true, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_true, y_pred))

mae_val = None
rmse_val = None
# ===============================================================

In [14]:
df_test = h(df_test)

In [15]:
# Nota: df_test tiene que tener la columna 'prediction'
ev = Evaluator(df_test, MODEL_NAME)
report = ev.getReport()
ev.visualEvaluation()

## Registro del modelo en MLflow

In [18]:
mlflow.set_tracking_uri("file:./mlflow_experiments")
mlflow.set_experiment("takeoff_time_prediction")

with mlflow.start_run():

    # - Datos generales -

    # ========================================================================
    mlflow.set_tag("model_type", MODEL_NAME)
    mlflow.set_tag("framework", "pandas") # scikit-learn, tensorflow, etc.
    mlflow.set_tag("target_variable", "takeoff_time") # variable respuesta
    mlflow.set_tag("preprocessing", "none") # transformaciones separadas por un +
    mlflow.set_tag("dataset", "original") # indicar si se ha modificado el conjunto de datos
    mlflow.set_tag("seed", SEED) # semilla para replicabilidad
    # ========================================================================
    
    
    # - Métricas -

    mlflow.log_metric("execution_time_s", execution_time)

    mlflow.log_metric("mae_train", mae_train)
    mlflow.log_metric("rmse_train", rmse_train)

    # Registrar métricas globales en test
    for metric_name, value in report["global"].items():
        mlflow.log_metric(f"{metric_name}_test", value)
    
    # Registrar métricas por runway
    for runway, metrics in report["by_runway"].items():
        for metric_name, value in metrics.items():
            mlflow.log_metric(f"{metric_name}_test_runway_{runway}", value)
    
    # Registrar métricas por holding point
    for hp, metrics in report["by_holding_point"].items():
        for metric_name, value in metrics.items():
            mlflow.log_metric(f"{metric_name}_test_hp_{hp}", value)

    # - Modelo -

    import mlflow.pyfunc

    class HeuristicModel(mlflow.pyfunc.PythonModel):
        
        def predict(self, context, model_input):
            # model_input será un DataFrame
            return model_input.apply(h, axis=1)
    
    model = HeuristicModel()
    mlflow.pyfunc.log_model(
        artifact_path=MODEL_NAME,
        python_model=model
    )
    

/opt/anaconda3/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning:

Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.

2025/04/26 19:31:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
# - Visualizar experimentos -
# !mlflow ui --backend-store-uri ./mlflow_experiments